In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import glob, os, re, sys, holidays, datetime

In [2]:
cluster_df = pd.read_csv("./ap_cluster_df.csv", dtype={'n_cluster_pm10':int, 'n_cluster_pm25':int})
cluster_df.columns = ["areacode", "pm10grp", "pm25grp"]
cluster_df
#pd.melt(cluster_df, id_vars=['areacode'], 
#                     value_vars=cluster_df.columns)

,areacode,pm10grp,pm25grp
0,11110,2,2
1,11140,2,2
2,11170,2,4
3,11200,2,4
4,11215,2,2
...,...,...,...
244,48870,0,6
245,48880,5,6
246,48890,5,6
247,50110,6,6


### NEDIS database ICD10 I63
- I60/I61은 응급실 내원 환자의 수를 7, 8개 cluster로 나누어 매일 응급실 방문자의 수를 세어보면 10명 이하로 온 날이나 아예 존재하지 않는 cluster가 있어서 연구에서 제외하고 환자의 수가 충분한 I63에 대해서만 연구를 진행

In [3]:
I63 = pd.read_csv("../I63mainDiag.csv", low_memory=False)
# 세종특별자치시 자료는 연구에서 제외
I63 = I63[I63['PTMIGUCD']!=36110]
# 인천시 남구 -> 인천시 미추홀구
I63.replace({'PTMIGUCD': 28170}, 28177, inplace=True)
# 경기도 부천시 구(원미, 소사 등) 들이 부천시 하나로 통합
I63.replace({'PTMIGUCD': [41195, 41197, 41199]}, 41190, inplace=True)

# adding universial ID
I63['UID'] = I63['UID'].apply(lambda x: 1000000+int(x))
# converting object to datetime 
I63.loc[:, 'adm_ED_date'] = pd.to_datetime(I63['PTMIINDT'].astype(str)+" "+I63['PTMIINTM'].astype(str), 
                                           format='%Y%m%d %H%M', errors = 'coerce')
I63.loc[:, 'onset_date'] = pd.to_datetime(I63['PTMIAKDT'].astype(str)+" "+I63['PTMIAKTM'].astype(str), 
                                           format='%Y%m%d %H%M', errors = 'coerce')
I63.loc[:,'I63daily_adm'] = I63['adm_ED_date'].dt.date
I63.loc[:, 'I63daily_onset'] = I63['onset_date'].dt.date
# removing the patients visit data when the patient's address is not known
I63 = I63[I63['PTMIGUCD'].notnull()]
I63.loc[:, 'PTMIGUCD'] = I63.loc[:, 'PTMIGUCD'].apply(lambda x: int(x) if x!=np.nan else np.nan)
# merging I63 with the cluster_df
I63 = pd.merge(I63, cluster_df, how='left', left_on='PTMIGUCD', right_on='areacode')
#print(I63[I63['pm10grp'].isnull()])
I63.loc[:, 'pm10grp'] = I63.loc[:, 'pm10grp'].apply(lambda x: str(int(x)))
I63.loc[:, 'pm25grp'] = I63.loc[:, 'pm25grp'].apply(lambda x: str(int(x)))
I63_daily_pm10grp = I63.set_index('adm_ED_date').groupby('pm10grp')\
                           .resample('D').agg({'I63daily_adm':'count', 'I63daily_onset':'count'})\
                           .unstack(level=[0])
I63_daily_pm25grp = I63.set_index('adm_ED_date').groupby('pm25grp')\
                           .resample('D').agg({'I63daily_adm':'count', 'I63daily_onset':'count'})\
                           .unstack(level=[0])
print(I63_daily_pm10grp.head())
I63_daily_pm25grp.head()

            I63daily_adm                           I63daily_onset            \
pm10grp                0  1   2   3   4   5  6   7              0  1   2  3   
adm_ED_date                                                                   
2015-01-01            11  1  27   7  13  21  7   8             10  1  26  6   
2015-01-02            24  0  25  10  18  26  3  12             21  0  22  9   
2015-01-03            15  1  17   5  16  28  5   8             13  1  14  5   
2015-01-04            11  3  20   8  15  30  5  11             10  3  18  8   
2015-01-05            19  1  33  10  22  28  2  14             16  1  32  9   

                            
pm10grp       4   5  6   7  
adm_ED_date                 
2015-01-01    9  16  2   7  
2015-01-02   16  25  2  11  
2015-01-03   14  24  4   5  
2015-01-04   13  25  2   8  
2015-01-05   19  24  1  12  


I63daily_adm                      I63daily_onset                   \
pm25grp                0  1   2  3   4  5   6              0  1   2  3   4  5   
adm_ED_date                                                                     
2015-01-01            11  7  27  3  21  4  22             11  2  26  1  17  3   
2015-01-02            24  2  28  4  23  7  30             23  1  22  3  22  6   
2015-01-03            16  4  18  3  19  7  28             14  3  15  3  16  5   
2015-01-04             9  4  24  6  19  9  32              8  1  21  6  18  6   
2015-01-05            19  2  38  1  34  6  29             16  1  38  1  29  4   

                 
pm25grp       6  
adm_ED_date      
2015-01-01   17  
2015-01-02   29  
2015-01-03   24  
2015-01-04   27  
2015-01-05   25

In [4]:
I63_daily_pm10grp.columns.names[1]

'pm10grp'

In [5]:
headers = [h[0]+"_"+I63_daily_pm10grp.columns.names[1]+h[1] \
           for h in I63_daily_pm10grp.columns]
I63_daily_pm10grp.columns = headers
headers = [h[0]+"_"+I63_daily_pm25grp.columns.names[1]+h[1] \
           for h in I63_daily_pm25grp.columns]
I63_daily_pm25grp.columns = headers
I63_daily_pm10grp.head()
I63_daily_pm25grp.head()

,I63daily_adm_pm25grp0,I63daily_adm_pm25grp1,I63daily_adm_pm25grp2,I63daily_adm_pm25grp3,I63daily_adm_pm25grp4,I63daily_adm_pm25grp5,I63daily_adm_pm25grp6,I63daily_onset_pm25grp0,I63daily_onset_pm25grp1,I63daily_onset_pm25grp2,I63daily_onset_pm25grp3,I63daily_onset_pm25grp4,I63daily_onset_pm25grp5,I63daily_onset_pm25grp6
adm_ED_date,,,,,,,,,,,,,,
2015-01-01,11,7,27,3,21,4,22,11,2,26,1,17,3,17
2015-01-02,24,2,28,4,23,7,30,23,1,22,3,22,6,29
2015-01-03,16,4,18,3,19,7,28,14,3,15,3,16,5,24
2015-01-04,9,4,24,6,19,9,32,8,1,21,6,18,6,27
2015-01-05,19,2,38,1,34,6,29,16,1,38,1,29,4,25


In [6]:
# I63 sex group
I63_daily_sex_pm10grp = I63.set_index('adm_ED_date')\
                           .groupby(['PTMISEXX', 'pm10grp'])\
                           .resample('D').agg({'I63daily_adm':'count'}).unstack(level=[0, 1])
I63_daily_sex_pm25grp = I63.set_index('adm_ED_date')\
                           .groupby(['PTMISEXX', 'pm25grp'])\
                           .resample('D').agg({'I63daily_adm':'count'}).unstack(level=[0, 1])
print(I63_daily_sex_pm10grp.head())
print(I63_daily_sex_pm25grp.head())

            I63daily_adm                                                  \
PTMISEXX               F                                          M        
pm10grp                0    1     2    3     4     5    6    7    0    1   
adm_ED_date                                                                
2015-01-01           8.0  1.0  11.0  2.0   7.0   7.0  4.0  5.0  3.0  NaN   
2015-01-02          16.0  0.0   7.0  2.0   6.0  13.0  1.0  7.0  8.0  NaN   
2015-01-03           8.0  0.0   8.0  3.0   8.0   9.0  1.0  5.0  7.0  1.0   
2015-01-04           4.0  3.0   5.0  6.0   7.0  12.0  3.0  3.0  7.0  0.0   
2015-01-05          10.0  1.0  12.0  5.0  13.0  15.0  1.0  5.0  9.0  0.0   

                                              
PTMISEXX                                      
pm10grp         2    3     4     5    6    7  
adm_ED_date                                   
2015-01-01   16.0  5.0   6.0  14.0  3.0  3.0  
2015-01-02   18.0  8.0  12.0  13.0  2.0  5.0  
2015-01-03    9.0  2.0   8.0  19.

In [7]:
headers = [h[0]+"_"+h[1]+"_"+I63_daily_sex_pm10grp.columns.names[2]+h[2] \
           for h in I63_daily_sex_pm10grp.columns]
I63_daily_sex_pm10grp.columns = headers
headers = [h[0]+"_"+h[1]+"_"+I63_daily_sex_pm25grp.columns.names[2]+h[2] \
           for h in I63_daily_sex_pm25grp.columns]
I63_daily_sex_pm25grp.columns = headers
I63_daily_sex_pm25grp.head()

,I63daily_adm_F_pm25grp0,I63daily_adm_F_pm25grp1,I63daily_adm_F_pm25grp2,I63daily_adm_F_pm25grp3,I63daily_adm_F_pm25grp4,I63daily_adm_F_pm25grp5,I63daily_adm_F_pm25grp6,I63daily_adm_M_pm25grp0,I63daily_adm_M_pm25grp1,I63daily_adm_M_pm25grp2,I63daily_adm_M_pm25grp3,I63daily_adm_M_pm25grp4,I63daily_adm_M_pm25grp5,I63daily_adm_M_pm25grp6
adm_ED_date,,,,,,,,,,,,,,
2015-01-01,9.0,4.0,13.0,3.0,7.0,2.0,7.0,2.0,3.0,14.0,NaN,14.0,2.0,15.0
2015-01-02,16.0,0.0,9.0,1.0,5.0,5.0,16.0,8.0,2.0,19.0,3.0,18.0,2.0,14.0
2015-01-03,8.0,1.0,9.0,1.0,10.0,4.0,9.0,8.0,3.0,9.0,2.0,9.0,3.0,19.0
2015-01-04,2.0,2.0,9.0,3.0,11.0,3.0,13.0,7.0,2.0,15.0,3.0,8.0,6.0,19.0
2015-01-05,9.0,1.0,17.0,0.0,16.0,3.0,16.0,10.0,1.0,21.0,1.0,18.0,3.0,13.0


In [8]:
# I63 age group
I63['agegrp'] = pd.cut(x=I63['PTMIBRTD'], bins=[0, 13, 26], labels=['Y', 'O'])
I63_daily_age_pm10grp = I63.set_index('adm_ED_date')\
                           .groupby(['agegrp', 'pm10grp'])\
                           .resample('D').agg({'I63daily_adm':'count'}).unstack(level=[0, 1])
I63_daily_age_pm25grp = I63.set_index('adm_ED_date')\
                           .groupby(['agegrp', 'pm25grp'])\
                           .resample('D').agg({'I63daily_adm':'count'}).unstack(level=[0, 1])
headers = [h[0]+"_"+h[1]+"_"+I63_daily_age_pm10grp.columns.names[2]+h[2]\
           for h in I63_daily_age_pm10grp.columns]
I63_daily_age_pm10grp.columns = headers
headers = [h[0]+"_"+h[1]+"_"+I63_daily_age_pm25grp.columns.names[2]+h[2] \
           for h in I63_daily_age_pm25grp.columns]
I63_daily_age_pm25grp.columns = headers
I63_daily_age_pm25grp.head()

,I63daily_adm_Y_pm25grp0,I63daily_adm_Y_pm25grp1,I63daily_adm_Y_pm25grp2,I63daily_adm_Y_pm25grp3,I63daily_adm_Y_pm25grp4,I63daily_adm_Y_pm25grp5,I63daily_adm_Y_pm25grp6,I63daily_adm_O_pm25grp0,I63daily_adm_O_pm25grp1,I63daily_adm_O_pm25grp2,I63daily_adm_O_pm25grp3,I63daily_adm_O_pm25grp4,I63daily_adm_O_pm25grp5,I63daily_adm_O_pm25grp6
adm_ED_date,,,,,,,,,,,,,,
2015-01-01,2.0,NaN,7.0,NaN,2.0,1.0,8.0,9.0,7.0,20.0,3.0,19.0,3.0,14.0
2015-01-02,3.0,NaN,5.0,1.0,10.0,3.0,5.0,21.0,2.0,23.0,3.0,13.0,4.0,25.0
2015-01-03,4.0,NaN,5.0,2.0,7.0,1.0,6.0,12.0,4.0,13.0,1.0,12.0,6.0,22.0
2015-01-04,2.0,1.0,4.0,0.0,5.0,2.0,9.0,7.0,3.0,20.0,6.0,14.0,7.0,23.0
2015-01-05,5.0,0.0,8.0,0.0,9.0,1.0,11.0,14.0,2.0,30.0,1.0,25.0,5.0,18.0


In [9]:
I63pm10grp = pd.concat([I63_daily_pm10grp, I63_daily_sex_pm10grp, I63_daily_age_pm10grp], axis=1)
I63pm10grp.columns
I63pm25grp = pd.concat([I63_daily_pm25grp, I63_daily_sex_pm25grp, I63_daily_age_pm25grp], axis=1)
I63pm10grp.head()

,I63daily_adm_pm10grp0,I63daily_adm_pm10grp1,I63daily_adm_pm10grp2,I63daily_adm_pm10grp3,I63daily_adm_pm10grp4,I63daily_adm_pm10grp5,I63daily_adm_pm10grp6,I63daily_adm_pm10grp7,I63daily_onset_pm10grp0,I63daily_onset_pm10grp1,...,I63daily_adm_Y_pm10grp6,I63daily_adm_Y_pm10grp7,I63daily_adm_O_pm10grp0,I63daily_adm_O_pm10grp1,I63daily_adm_O_pm10grp2,I63daily_adm_O_pm10grp3,I63daily_adm_O_pm10grp4,I63daily_adm_O_pm10grp5,I63daily_adm_O_pm10grp6,I63daily_adm_O_pm10grp7
adm_ED_date,,,,,,,,,,,,,,,,,,,,,
2015-01-01,11,1,27,7,13,21,7,8,10,1,...,NaN,1.0,9.0,1.0,21.0,6.0,11.0,13.0,7.0,7.0
2015-01-02,24,0,25,10,18,26,3,12,21,0,...,NaN,3.0,22.0,0.0,17.0,7.0,12.0,21.0,3.0,9.0
2015-01-03,15,1,17,5,16,28,5,8,13,1,...,1.0,1.0,11.0,1.0,10.0,5.0,10.0,22.0,4.0,7.0
2015-01-04,11,3,20,8,15,30,5,11,10,3,...,1.0,1.0,8.0,3.0,17.0,6.0,11.0,21.0,4.0,10.0
2015-01-05,19,1,33,10,22,28,2,14,16,1,...,0.0,5.0,15.0,1.0,26.0,7.0,18.0,17.0,2.0,9.0


In [10]:
I63pm10grp.fillna(0, inplace=True)
I63pm10grp.index.name = 'date'
print(I63pm10grp.index.name)
I63pm25grp.fillna(0, inplace=True)
I63pm25grp.index.name = 'date'
print(I63pm25grp.index.name)

date
date


In [11]:
# I63pm25grp = pd.melt(I63_daily_adm_pm25grp.reset_index(), id_vars=['adm_ED_date'], 
#                      value_vars=I63_daily_adm_pm25grp.columns,
#                      var_name='grp', value_name='N_I63')

# I63pm10grp = pd.melt(I63_daily_adm_pm10grp.reset_index(), id_vars=['adm_ED_date'], 
#                      value_vars=I63_daily_adm_pm10grp.columns,
#                      var_name='grp', value_name='N_I63')
# I63grps = pd.concat([I63pm25grp, I63pm10grp], axis=0, ignore_index=False)
# I63grps[I63grps['adm_ED_date']=='2015-1-1']

### Air pollutant data

In [12]:
apfiles = glob.glob("../../../../case_crossover/interpolate_aps/interpolated/finals/aps201[4-9]*csv")
apfiles.sort()
apfiles

['../../../../case_crossover/interpolate_aps/interpolated/finals/aps2014_kostat_code.csv',
 '../../../../case_crossover/interpolate_aps/interpolated/finals/aps2015_kostat_code.csv',
 '../../../../case_crossover/interpolate_aps/interpolated/finals/aps2016_kostat_code.csv',
 '../../../../case_crossover/interpolate_aps/interpolated/finals/aps2017_kostat_code.csv',
 '../../../../case_crossover/interpolate_aps/interpolated/finals/aps2018_kostat_code.csv',
 '../../../../case_crossover/interpolate_aps/interpolated/finals/aps2019_kostat_code.csv']

In [13]:
for i, apfile in enumerate(apfiles) :
    if i == 0 :
        ap = pd.read_csv(apfile)
    else :
        tmp = pd.read_csv(apfile)
        ap = ap.append(tmp, ignore_index=True)
ap['date'] = pd.to_datetime(ap['date'])
ap = ap.sort_values(by='date')

ap = ap[(ap['date'] >= '2014-12-18') & (ap['date']< '2019-1-15')]
ap = pd.merge(ap, cluster_df, how='left', on='areacode')
ap.loc[:, 'pm10grp'] = ap.loc[:, 'pm10grp'].apply(lambda x: str(int(x)))
ap.loc[:, 'pm25grp'] = ap.loc[:, 'pm25grp'].apply(lambda x: str(int(x)))

In [14]:
ap_daily_pm10grp = ap.set_index('date')\
                     .groupby('pm10grp')\
                     .resample('D')\
                     .agg({'SO2':'mean','CO':'mean', 'O3':'mean', 
                           'NO2':'mean', 'PM10':'mean', 'PM25':'mean'}).unstack(level=[0])
ap_daily_pm25grp = ap.set_index('date')\
                     .groupby('pm25grp')\
                     .resample('D')\
                     .agg({'SO2':'mean','CO':'mean', 'O3':'mean',
                           'NO2':'mean','PM10':'mean', 'PM25':'mean'}).unstack(level=[0])
print(ap_daily_pm10grp.head())
print(ap_daily_pm25grp.tail())

                 SO2                                                    \
pm10grp            0         1         2         3         4         5   
date                                                                     
2014-12-18  0.004593  0.005287  0.005651  0.006168  0.006344  0.003899   
2014-12-19  0.005787  0.005343  0.006408  0.006521  0.007416  0.006507   
2014-12-20  0.005057  0.005700  0.005578  0.005869  0.005862  0.005353   
2014-12-21  0.004254  0.004888  0.005287  0.005397  0.005646  0.003943   
2014-12-22  0.004593  0.005075  0.006257  0.006087  0.006949  0.004667   

                                      CO            ...       PM10             \
pm10grp            6         7         0         1  ...          6          7   
date                                                ...                         
2014-12-18  0.003780  0.005349  0.483189  0.491379  ...  18.180779  23.674590   
2014-12-19  0.005581  0.006017  0.766425  0.673333  ...  32.733819  47.759011   
20

In [15]:
headers = [h[0]+"_"+ap_daily_pm10grp.columns.names[1]+h[1] \
           for h in ap_daily_pm10grp.columns]
ap_daily_pm10grp.columns = headers

headers = [h[0]+"_"+ap_daily_pm25grp.columns.names[1]+h[1] \
           for h in ap_daily_pm25grp.columns]
ap_daily_pm25grp.columns = headers
ap_daily_pm25grp.head()

,SO2_pm25grp0,SO2_pm25grp1,SO2_pm25grp2,SO2_pm25grp3,SO2_pm25grp4,SO2_pm25grp5,SO2_pm25grp6,CO_pm25grp0,CO_pm25grp1,CO_pm25grp2,...,PM10_pm25grp4,PM10_pm25grp5,PM10_pm25grp6,PM25_pm25grp0,PM25_pm25grp1,PM25_pm25grp2,PM25_pm25grp3,PM25_pm25grp4,PM25_pm25grp5,PM25_pm25grp6
date,,,,,,,,,,,,,,,,,,,,,
2014-12-18,0.004710,0.003869,0.005740,0.006466,0.005666,0.005582,0.003964,0.489879,0.431811,0.534353,...,27.580569,26.506783,22.607386,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014-12-19,0.005668,0.005325,0.006478,0.007386,0.006611,0.005472,0.006598,0.761457,0.753853,0.801824,...,49.290042,60.789801,45.107890,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014-12-20,0.005042,0.004405,0.005869,0.005920,0.005531,0.005710,0.005387,0.624923,0.630609,0.613725,...,32.699501,41.924084,45.160248,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014-12-21,0.004304,0.003766,0.005264,0.005530,0.005276,0.004972,0.003950,0.464501,0.407399,0.510022,...,22.712528,22.933540,21.695949,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014-12-22,0.004695,0.004536,0.005878,0.007640,0.006166,0.005488,0.004638,0.591572,0.517817,0.676779,...,32.943301,34.745438,29.663224,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
# aps_pm25grp = pd.melt(aps_daily_pm25grp.reset_index(), id_vars=['date'], 
#                      value_vars=aps_daily_pm25grp.columns,
#                      var_name='aps_grp', value_name='aps_values')

# aps_pm10grp = pd.melt(aps_daily_pm10grp.reset_index(), id_vars=['date'], 
#                      value_vars=aps_daily_pm10grp.columns,
#                      var_name='aps_grp', value_name='aps_values')
# aps_grps = pd.concat([aps_pm25grp, aps_pm10grp], axis=0, ignore_index=False)

### Meteorological data

In [17]:
met_path = os.path.join("/home/yihahn/data/nas125/epidemiology/nedis_stroke/", 
                       "case_crossover/interpolate_mets/interpolated/")
metfiles = glob.glob(os.path.join(met_path, "met201[4-9]*corrected.csv"))
metfiles.sort()

In [18]:
for i, metfile in enumerate(metfiles) :
    if i == 0 :
        met = pd.read_csv(metfile, dtype={'T':float, 'H':float, 
                                         'precip':float, 'hPa':float, 'ws':float})
    else :
        tmp = pd.read_csv(metfile, dtype={'T':float, 'H':float, 
                                         'precip':float, 'hPa':float, 'ws':float})
        met = met.append(tmp, ignore_index=True)
met['date'] = pd.to_datetime(met['date'], errors='coerce')
met = met.sort_values(by='date')
met = met[(met['date'] >= '2014-12-18') & (met['date']< '2019-1-15')]
met = pd.merge(met, cluster_df, how='left', left_on='acode', right_on='areacode')
# 세종특별자치시 자료 제거
met = met[met['pm10grp'].notnull()]
met.loc[:, 'pm10grp'] = met.loc[:, 'pm10grp'].apply(lambda x: str(int(x)))
met.loc[:, 'pm25grp'] = met.loc[:, 'pm25grp'].apply(lambda x: str(int(x)))

In [19]:
dT = met.set_index('date').groupby('areacode').resample('D').agg({'T':('max', 'min')})
dT.loc[:, ('T', 'dT')] = dT.loc[:, ('T', 'max')] - dT.loc[:, ('T', 'min')]
dT.drop(columns=[('T', 'max'), ('T', 'min')], inplace = True)
dT.columns = [h[1] for h in dT.columns]
dT.reset_index(inplace=True)
dT['areacode'] = dT['areacode'].apply(lambda x: int(x))
dT = pd.merge(dT, cluster_df, how='left', on='areacode')
dT_daily_pm10grp = dT.set_index('date')\
                     .groupby('pm10grp')\
                     .resample('D')\
                     .agg({'dT':'mean'}).unstack(level=[0])
dT_daily_pm25grp = dT.set_index('date')\
                     .groupby('pm25grp')\
                     .resample('D')\
                     .agg({'dT':'mean'}).unstack(level=[0])
headers = [h[0]+"_pm10grp"+str(h[1]) for h in dT_daily_pm10grp.columns]
dT_daily_pm10grp.columns = headers
headers = [h[0]+"_pm25grp"+str(h[1]) for h in dT_daily_pm25grp.columns]
dT_daily_pm25grp.columns = headers

In [20]:
met_daily_pm10grp = met.set_index('date').groupby('pm10grp').resample('D').agg({
                       'T':'mean', 'H':'mean', 'precip':'sum','ws':'mean', 'hPa':'mean'}).unstack(level=[0])
met_daily_pm25grp = met.set_index('date').groupby('pm25grp').resample('D').agg({
                       'T':'mean', 'H':'mean', 'precip':'sum','ws':'mean', 'hPa':'mean'}).unstack(level=[0])
print(met_daily_pm10grp.head())
print(met_daily_pm25grp.tail())

                   T                                                     \
pm10grp            0          1         2         3         4         5   
date                                                                      
2014-12-18 -5.019525 -11.362220 -8.788068 -8.213536 -7.635288 -2.635301   
2014-12-19 -2.842175  -7.621002 -4.642835 -5.127124 -3.651727 -0.113312   
2014-12-20  1.169491  -2.116800 -1.277395 -1.081555 -0.715553  3.100298   
2014-12-21 -3.138508  -8.685639 -6.536667 -6.134960 -5.464890 -0.892255   
2014-12-22 -2.668659  -8.429356 -5.619633 -6.050862 -4.826157 -0.258572   

                                        H             ...        ws            \
pm10grp            6         7          0          1  ...         6         7   
date                                                  ...                       
2014-12-18 -2.415981 -8.134209  56.101063  50.239005  ...  3.299593  1.946161   
2014-12-19 -0.791449 -5.653151  70.695955  65.670277  ...  1.036214  1.0769

In [21]:
headers = [h[0]+"_" + met_daily_pm10grp.columns.names[1]+h[1] \
           for h in met_daily_pm10grp.columns]
met_daily_pm10grp.columns = headers
headers = [h[0]+"_"+ met_daily_pm25grp.columns.names[1]+h[1] \
           for h in met_daily_pm25grp.columns]
met_daily_pm25grp.columns = headers

In [22]:
met_daily_pm10grp = pd.merge(met_daily_pm10grp, dT_daily_pm10grp, on = 'date')
met_daily_pm25grp = pd.merge(met_daily_pm25grp, dT_daily_pm25grp, on = 'date')
print(met_daily_pm10grp.head())
print(met_daily_pm25grp.head())

            T_pm10grp0  T_pm10grp1  T_pm10grp2  T_pm10grp3  T_pm10grp4  \
date                                                                     
2014-12-18   -5.019525  -11.362220   -8.788068   -8.213536   -7.635288   
2014-12-19   -2.842175   -7.621002   -4.642835   -5.127124   -3.651727   
2014-12-20    1.169491   -2.116800   -1.277395   -1.081555   -0.715553   
2014-12-21   -3.138508   -8.685639   -6.536667   -6.134960   -5.464890   
2014-12-22   -2.668659   -8.429356   -5.619633   -6.050862   -4.826157   

            T_pm10grp5  T_pm10grp6  T_pm10grp7  H_pm10grp0  H_pm10grp1  ...  \
date                                                                    ...   
2014-12-18   -2.635301   -2.415981   -8.134209   56.101063   50.239005  ...   
2014-12-19   -0.113312   -0.791449   -5.653151   70.695955   65.670277  ...   
2014-12-20    3.100298    2.442189   -0.593757   70.643786   61.625834  ...   
2014-12-21   -0.892255   -0.939324   -5.351757   66.624531   66.943897  ...   
2014-12

In [23]:
print(I63pm10grp.shape, I63pm25grp.shape, ap_daily_pm10grp.shape, ap_daily_pm25grp.shape, 
      met_daily_pm10grp.shape, met_daily_pm25grp.shape)

(1461, 48) (1461, 42) (1489, 48) (1489, 42) (1489, 48) (1489, 42)


In [24]:
mers_path = os.path.join("/home/yihahn/data/nas125",
                         "epidemiology/nedis_stroke/case_crossover/")
mers = pd.read_csv(os.path.join(mers_path, "mers.csv"))
mers['date'] = pd.to_datetime(mers['date'])
mers.columns = ['date', 'mers']
mers.set_index('date', inplace=True)

In [25]:
I63grp_df = pd.concat([I63pm10grp, I63pm25grp, ap_daily_pm10grp, ap_daily_pm25grp, 
                       met_daily_pm10grp, met_daily_pm25grp, mers], axis=1)
print(I63grp_df.shape)
I63grp_df.head()

(1489, 271)


,I63daily_adm_pm10grp0,I63daily_adm_pm10grp1,I63daily_adm_pm10grp2,I63daily_adm_pm10grp3,I63daily_adm_pm10grp4,I63daily_adm_pm10grp5,I63daily_adm_pm10grp6,I63daily_adm_pm10grp7,I63daily_onset_pm10grp0,I63daily_onset_pm10grp1,...,hPa_pm25grp5,hPa_pm25grp6,dT_pm25grp0,dT_pm25grp1,dT_pm25grp2,dT_pm25grp3,dT_pm25grp4,dT_pm25grp5,dT_pm25grp6,mers
date,,,,,,,,,,,,,,,,,,,,,
2014-12-18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1023.448461,1020.801895,9.145861,6.944138,10.816958,8.243888,9.789794,9.185998,9.164488,NaN
2014-12-19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1020.311560,1020.265293,11.593395,11.930265,10.305166,7.895833,8.666217,10.059204,12.038754,NaN
2014-12-20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1011.668627,1009.449816,6.120563,4.279126,7.979006,8.528554,7.120783,6.029275,7.940950,NaN
2014-12-21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1013.845610,1012.339520,7.028354,4.795995,8.781909,5.291161,6.981801,8.471962,6.325915,NaN
2014-12-22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1014.885682,1013.487111,9.932999,7.129364,11.149242,8.834836,10.812363,12.080325,9.017408,NaN


In [26]:
# merging two dictionaries  
def merge_two_dicts(x, y):
    """Given two dictionaries, merge them into a new dict as a shallow copy."""
    z = x.copy()
    z.update(y)
    return z

In [27]:
# collecting all public holidays during the study period
holidays_kor = {}
for year in [2015, 2016, 2017, 2018] :
    dic = holidays.KR(years=year)
    if year == 2017 :
        dic[datetime.date(2017, 10, 6)] = 'Alternative holiday of Chuseok'
    else :
        holidays_kor = merge_two_dicts(holidays_kor, dic)

In [28]:
def baking_holitrue(d) :
    #print(d.year, type(d.date()))
    if d.weekday() < 5 :
        if datetime.date(d.year, d.month, d.day) in holidays_kor :
            return 2
        else :
            return 0
    elif d.weekday() == 5 :
        if datetime.date(d.year, d.month, d.day) in holidays_kor :
            return 2
        else :
            return 1
    else :
        return 2

In [29]:
def ldsph(x) :
    # Lapse days from previous holiday
    if x['holiTrue'] == 2 :
        return 0
    else :
        for i in range(1, 7, 1) :
            theday = x.name - pd.Timedelta(i, unit='d')
            if theday.weekday() == 6 or \
            datetime.date(theday.year, theday.month, theday.day) in holidays_kor: 
                ldsph = (x.name - theday) / np.timedelta64(1, 'D')
                return ldsph 

In [30]:
I63grp_df['dow'] = I63grp_df.index.weekday
I63grp_df['mon'] = I63grp_df.index.month
I63grp_df['t'] = range(1, I63grp_df.shape[0]+1, 1)
I63grp_df['holiTrue'] = I63grp_df.apply(lambda x: baking_holitrue(x.name), axis=1)
I63grp_df['ldsph'] = I63grp_df.apply(lambda x: ldsph(x), axis=1)

In [31]:
I63grp_df.columns

Index(['I63daily_adm_pm10grp0', 'I63daily_adm_pm10grp1',
       'I63daily_adm_pm10grp2', 'I63daily_adm_pm10grp3',
       'I63daily_adm_pm10grp4', 'I63daily_adm_pm10grp5',
       'I63daily_adm_pm10grp6', 'I63daily_adm_pm10grp7',
       'I63daily_onset_pm10grp0', 'I63daily_onset_pm10grp1',
       ...
       'dT_pm25grp3', 'dT_pm25grp4', 'dT_pm25grp5', 'dT_pm25grp6', 'mers',
       'dow', 'mon', 't', 'holiTrue', 'ldsph'],
      dtype='object', length=276)

In [32]:
ap_cluster_path = "ap_cluster_grp"
os.makedirs(ap_cluster_path, exist_ok=True)

In [33]:
for grp_head in ['pm10grp', 'pm25grp'] :
    for j in np.sort(cluster_df[grp_head].unique()) :
        grp = grp_head + str(j)
        include_columns = ['mers', 'dow', 'mon', 't', 'holiTrue', 'ldsph']
        for column in I63grp_df.columns :
            pattern = re.compile(grp)
            if pattern.findall(column) :
                include_columns.append(column)
        out_path = os.path.join(ap_cluster_path, grp+".csv")
        I63grp_df[include_columns].to_csv(out_path, index_label=None)